In [1]:
from altair.vegalite.v4.schema.core import InlineData
from numpy.core.numeric import outer
import pandas as pd
import numpy as np
data = pd.read_csv("http://web.stanford.edu/~hastie/ElemStatLearn/datasets/SAheart.data")

In [2]:
heart = data.iloc[0:data.shape[0] , 1:data.shape[1]]
#%%
heart['famhist'] = np.where(heart['famhist'] == 'Present',1, 0 )
#%%
X= heart.drop(columns='chd')
X=X.to_numpy()
y=np.array(heart.chd)

In [3]:
import xgboost as xgb
from xgboost.sklearn import XGBClassifier  
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split, GridSearchCV
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.pyplot import rcParams
rcParams['figure.figsize'] = 12, 4

In [4]:
#Here we use the cross validation function of xgb. we give the function an algorithm ( the xgb classifier), the train dataframe, the predictors
# TODO: need to integrate teh test set in there too!! (inside the function fo the split)  
def modelfit(alg, dtrain, y, useTrainCV=True, cv_folds=5, early_stopping_rounds=50):
    
    if useTrainCV:
        print('using train cv')
        xgb_param = alg.get_xgb_params()
        xgtrain = xgb.DMatrix(dtrain, label=y)
        cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=alg.get_params()['n_estimators'], nfold=cv_folds,
            metrics='auc', early_stopping_rounds=early_stopping_rounds, show_progress=True)
        alg.set_params(n_estimators=cvresult.shape[0])
    
    #Fit the algorithm on the data
    alg.fit(dtrain, y, eval_metric='auc')
    print('data fitted')    
    #Predict training set:
    dtrain_predictions = alg.predict(dtrain)
    dtrain_predprob = alg.predict_proba(dtrain)[:,1]
    #TODO: not sure what the above does, what is the pred_prob?

    #Print model report:
    print (f"\nModel Report\nAccuracy : {accuracy_score(y, dtrain_predictions)}\nAUC Score (Train): {roc_auc_score(y, dtrain_predprob)}")
                    
    feat_imp = pd.Series(alg.booster().get_fscore()).sort_values(ascending=False)
    feat_imp.plot(kind='bar', title='Feature Importances')
    plt.ylabel('Feature Importance Score')

In [5]:
xgb0=XGBClassifier(learning_rate = 0.1,
n_estimators=1000,
max_depth=5, #max depth of a tree
min_child_weight=1, # weight of leaf? needs to be tuned too high values can lead to underfitting.
gamma=0,
subsample=0.8,
col_sample_bytree=0.8, #prop of obs to be used for each tree. 
objective='binary:logistic',
nthread=4, #used for parallel processing! enter number of cores in the system.
scale_pos_weight=1, # for high class imbalance!helps in fater convergence..
seed=27)

In [ ]:
#lets not use the function 
# modelfit(xgb0, X, y)
#we then first get the data matrix:
data_matrix=xgb.DMatrix(data=X, label=y)
#do the train/test split:
X_train, y_train, X_test, y_test = train_test_split(X, y, test_size=0.2, random_state=123) 
xgb0.fit(X_train,y_train, eval_metric='auc')
preds=xgb0.predict(X_test)
print (f"\nModel Report\nAccuracy : {accuracy_score(y, preds)}\nAUC Score (Train): {roc_auc_score(y, preds)}")